In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import os 
from sklearn.metrics import mean_absolute_percentage_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
# 1. Tentukan Lokasi File Input (Harus sama persis dengan Langkah 1)
path_file_input = r"E:\Skripsi\Prediksi-bawang-merah\python\data\Data_Clean_Surabaya.xlsx"

# 2. Baca File Excel
if os.path.exists(path_file_input):
    df = pd.read_excel(path_file_input)
    print("✓ Data berhasil dimuat dari:")
    print(f"   {path_file_input}")
    print(f"✓ Jumlah baris data: {len(df)}")
else:
    print("X FILE TIDAK DITEMUKAN!")
    print(f"   Cek apakah file ini ada: {path_file_input}")
    print("   (Pastikan Anda sudah menjalankan 'Save' di preprocessing.ipynb)")
raw_data = df['Harga (Rp)'].values.reshape(-1, 1)

print(f"Data siap digunakan. Jumlah baris: {len(raw_data)}")
print("Melanjutkan ke Uji Coba Skenario A...")

# DEFINISI FUNGSI & ARSITEKTUR (Sesuai Bab 3 Metodologi)

# 1. Fungsi Sliding Window (Bab 3.3.1)
def create_sliding_window(dataset, window_size):
    X, Y = [], []
    for i in range(len(dataset) - window_size):
        X.append(dataset[i:(i + window_size), 0])
        Y.append(dataset[i + window_size, 0])
    return np.array(X), np.array(Y)

# 2. Arsitektur LSTM (Bab 3.3.3 - Tabel 3.4) [cite: 364-368]
def build_model_lstm(input_shape):
    model = Sequential()
    # Layer 1: LSTM 100 unit + Dropout 0.02
    model.add(LSTM(100, return_sequences=True, input_shape=input_shape))
    model.add(Dropout(0.02))
    
    # Layer 2: LSTM 100 unit + Dropout 0.02
    model.add(LSTM(100, return_sequences=False))
    model.add(Dropout(0.02))
    
    # Layer 3: Dense 25 unit
    model.add(Dense(25, activation='relu'))
    
    # Layer Output: Dense 1 unit
    model.add(Dense(1)) 
    
    # Konfigurasi: Optimizer Adam (lr=0.001) & Loss MSE [cite: 373-374]
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
                  loss='mean_squared_error')
    return model

# EKSEKUSI UJI COBA SKENARIO A (VARIASI WINDOW SIZE)
# Skenario: Menguji Window Size 30, 60, dan 90.
# Parameter Tetap: Epoch 16, Batch Size 30.

scenario_windows = [30, 60, 90]
FIXED_EPOCH = 16      
FIXED_BATCH_SIZE = 30 

results = []

print("\n" + "="*50)
print("MULAI UJI COBA SKENARIO A (Mencari Window Size Terbaik)")
print("="*50)

for win_size in scenario_windows:
    print(f"\n[Proses] Sedang melatih model dengan Window Size: {win_size}...")
    
    # A. Transformasi Data (Potong data sesuai window size saat ini)
    X, Y = create_sliding_window(raw_data, win_size)
    
    # Reshape ke 3D [Samples, Timesteps, Features]
    X = np.reshape(X, (X.shape[0], X.shape[1], 1))
    
    # B. Pembagian Data (Split 80% Latih : 20% Uji) [cite: 352]
    train_size = int(len(X) * 0.8)
    X_train, X_test = X[:train_size], X[train_size:]
    y_train, y_test = Y[:train_size], Y[train_size:]
    
    # C. Pelatihan Model
    model = build_model_lstm((X_train.shape[1], 1))
    # Verbose=0 agar layar tidak penuh, kita hanya butuh hasil akhir
    model.fit(X_train, y_train, 
              epochs=FIXED_EPOCH, 
              batch_size=FIXED_BATCH_SIZE, 
              verbose=0)
    
    # D. Evaluasi (Hitung MAPE) [cite: 381]
    predictions = model.predict(X_test)
    
    # Hitung MAPE (Data Asli vs Prediksi)
    mape = mean_absolute_percentage_error(y_test, predictions) * 100
    
    print(f"   -> Selesai. Hasil MAPE: {mape:.4f}%")
    results.append({'Window Size': win_size, 'MAPE (%)': mape})

# HASIL AKHIR & KESIMPULAN
print("\n" + "="*50)
print("TABEL HASIL PERBANDINGAN SKENARIO A")
print("="*50)
results_df = pd.DataFrame(results)
print(results_df)

# Mencari nilai MAPE terendah otomatis
best_result = results_df.loc[results_df['MAPE (%)'].idxmin()]
best_window = int(best_result['Window Size'])

print("-" * 50)
print(f"KESIMPULAN TAHAP 1:")
print(f"Konfigurasi Window Size Terbaik adalah: {best_window}")
print(f"Dengan tingkat error (MAPE) terendah: {best_result['MAPE (%)']:.4f}%")
print("="*50)
print(f"Simpan nilai Window Size {best_window} ini untuk pengujian tahap berikutnya!")

✓ Data berhasil dimuat dari:
   E:\Skripsi\Prediksi-bawang-merah\python\data\Data_Clean_Surabaya.xlsx
✓ Jumlah baris data: 1043
Data siap digunakan. Jumlah baris: 1043
Melanjutkan ke Uji Coba Skenario A...

MULAI UJI COBA SKENARIO A (Mencari Window Size Terbaik)

[Proses] Sedang melatih model dengan Window Size: 30...
7/7 [==============================] - 9s 68ms/step
   -> Selesai. Hasil MAPE: 97.8657%

[Proses] Sedang melatih model dengan Window Size: 60...
7/7 [==============================] - 10s 82ms/step
   -> Selesai. Hasil MAPE: 98.0784%

[Proses] Sedang melatih model dengan Window Size: 90...
6/6 [==============================] - 4s 104ms/step
   -> Selesai. Hasil MAPE: 98.7165%

TABEL HASIL PERBANDINGAN SKENARIO A
   Window Size   MAPE (%)
0           30  97.865725
1           60  98.078370
2           90  98.716533
--------------------------------------------------
KESIMPULAN TAHAP 1:
Konfigurasi Window Size Terbaik adalah: 30
Dengan tingkat error (MAPE) terendah: 97.8657

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
import os
from sklearn.metrics import mean_absolute_percentage_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

# 1. LOAD DATA (DATA MENTAH)
path_file_input = r"E:\Skripsi\Prediksi-bawang-merah\python\data\Data_Clean_Surabaya.xlsx"

if os.path.exists(path_file_input):
    df = pd.read_excel(path_file_input)
    print("✓ Data berhasil dimuat.")
else:
    print("X File tidak ditemukan.")

# AMBIL DATA MENTAH (SESUAI RENCANA AWAL)
raw_data = df['Harga (Rp)'].values.reshape(-1, 1)

# KONFIGURASI DARI HASIL TAHAP 1

# Pemenang Tahap 1 adalah Window Size 90 (Sesuai Screenshot Anda)
FIXED_WINDOW_SIZE = 90  
FIXED_BATCH_SIZE = 30   # Masih default

# Variasi Epoch yang akan diuji sekarang (Sesuai Tabel 3.6)
scenario_epochs = [16, 30, 50]

# FUNGSI & MODEL

def create_sliding_window(dataset, window_size):
    X, Y = [], []
    for i in range(len(dataset) - window_size):
        X.append(dataset[i:(i + window_size), 0])
        Y.append(dataset[i + window_size, 0])
    return np.array(X), np.array(Y)

def build_model_lstm(input_shape):
    model = Sequential()
    model.add(LSTM(100, return_sequences=True, input_shape=input_shape))
    model.add(Dropout(0.02))
    model.add(LSTM(100, return_sequences=False))
    model.add(Dropout(0.02))
    model.add(Dense(25, activation='relu'))
    model.add(Dense(1))
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
                  loss='mean_squared_error')
    return model

# EKSEKUSI SKENARIO B (VARIASI EPOCH)


# Siapkan data dengan Window Size 90 (Pemenang Tahap 1)
print(f"\n[Info] Membentuk dataset dengan Window Size TETAP: {FIXED_WINDOW_SIZE}")
X, Y = create_sliding_window(raw_data, FIXED_WINDOW_SIZE)
X = np.reshape(X, (X.shape[0], X.shape[1], 1))

# Split Data 80:20
train_size = int(len(X) * 0.8)
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = Y[:train_size], Y[train_size:]

results = []

print("\n" + "="*60)
print("MULAI UJI COBA SKENARIO B (Mencari Epoch Terbaik)")
print("KONDISI: TANPA NORMALISASI")
print("="*60)

for epoch_count in scenario_epochs:
    print(f"\n[Proses] Training dengan Epoch: {epoch_count} ...")
    
    # Reset Model Baru
    model = build_model_lstm((X_train.shape[1], 1))
    
    # Training dengan variasi Epoch
    model.fit(X_train, y_train, 
              epochs=epoch_count,         
              batch_size=FIXED_BATCH_SIZE, 
              verbose=0)
    
    # Evaluasi
    predictions = model.predict(X_test)
    mape = mean_absolute_percentage_error(y_test, predictions) * 100
    
    print(f"   -> Selesai. Hasil MAPE: {mape:.4f}%")
    results.append({'Jumlah Epoch': epoch_count, 'MAPE (%)': mape})

# HASIL AKHIR SKENARIO B
print("\n" + "="*60)
results_df = pd.DataFrame(results)
print(results_df)

best_result = results_df.loc[results_df['MAPE (%)'].idxmin()]
best_epoch = int(best_result['Jumlah Epoch'])

print("-" * 60)
print(f"KESIMPULAN TAHAP 2:")
print(f"Epoch Terbaik adalah: {best_epoch}")
print(f"Dengan MAPE: {best_result['MAPE (%)']:.4f}%")
print("="*60)
print(f"Simpan Epoch {best_epoch} ini untuk tahap terakhir (Batch Size)!")

✓ Data berhasil dimuat.

[Info] Membentuk dataset dengan Window Size TETAP: 90

MULAI UJI COBA SKENARIO B (Mencari Epoch Terbaik)
KONDISI: TANPA NORMALISASI

[Proses] Training dengan Epoch: 16 ...
6/6 [==============================] - 5s 179ms/step
   -> Selesai. Hasil MAPE: 97.9391%

[Proses] Training dengan Epoch: 30 ...


In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
import os
from sklearn.metrics import mean_absolute_percentage_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

path_file_input = r"E:\Skripsi\Prediksi-bawang-merah\python\data\Data_Clean_Surabaya.xlsx"

if os.path.exists(path_file_input):
    df = pd.read_excel(path_file_input)
    print("✓ Data berhasil dimuat.")
else:
    print("X File tidak ditemukan.")

# AMBIL DATA MENTAH (KONDISI TANPA NORMALISASI)
raw_data = df['Harga (Rp)'].values.reshape(-1, 1)


# KONFIGURASI PEMENANG DARI TAHAP 1 & 2

FIXED_WINDOW_SIZE = 90  # Pemenang Tahap 1
FIXED_EPOCH = 50        # Pemenang Tahap 2 (Sesuai Screenshot Anda)

# Variasi Batch Size yang akan diuji sekarang (Sesuai Tabel 3.7)
scenario_batch_sizes = [16, 30, 64]


# FUNGSI & MODEL

def create_sliding_window(dataset, window_size):
    X, Y = [], []
    for i in range(len(dataset) - window_size):
        X.append(dataset[i:(i + window_size), 0])
        Y.append(dataset[i + window_size, 0])
    return np.array(X), np.array(Y)

def build_model_lstm(input_shape):
    model = Sequential()
    model.add(LSTM(100, return_sequences=True, input_shape=input_shape))
    model.add(Dropout(0.02))
    model.add(LSTM(100, return_sequences=False))
    model.add(Dropout(0.02))
    model.add(Dense(25, activation='relu'))
    model.add(Dense(1))
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
                  loss='mean_squared_error')
    return model

# =============================================================================
# EKSEKUSI SKENARIO C (VARIASI BATCH SIZE)
# =============================================================================

# Siapkan data dengan Window Size 90
print(f"\n[Info] Membentuk dataset dengan Window Size TETAP: {FIXED_WINDOW_SIZE}")
X, Y = create_sliding_window(raw_data, FIXED_WINDOW_SIZE)
X = np.reshape(X, (X.shape[0], X.shape[1], 1))

# Split Data 80:20
train_size = int(len(X) * 0.8)
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = Y[:train_size], Y[train_size:]

results = []

print("\n" + "="*60)
print("MULAI UJI COBA SKENARIO C (Mencari Batch Size Terbaik)")
print(f"Window: {FIXED_WINDOW_SIZE} | Epoch: {FIXED_EPOCH}")
print("KONDISI: TANPA NORMALISASI")
print("="*60)

for batch_count in scenario_batch_sizes:
    print(f"\n[Proses] Training dengan Batch Size: {batch_count} ...")
    
    # Reset Model
    model = build_model_lstm((X_train.shape[1], 1))
    
    # Training
    model.fit(X_train, y_train, 
              epochs=FIXED_EPOCH,          # Sudah Tetap
              batch_size=batch_count,      # Variabel yang diuji
              verbose=0)
    
    # Evaluasi
    predictions = model.predict(X_test)
    mape = mean_absolute_percentage_error(y_test, predictions) * 100
    
    print(f"   -> Selesai. Hasil MAPE: {mape:.4f}%")
    results.append({'Batch Size': batch_count, 'MAPE (%)': mape})

# HASIL AKHIR KESELURUHAN (TANPA NORMALISASI)

print("\n" + "="*60)
results_df = pd.DataFrame(results)
print(results_df)

best_result = results_df.loc[results_df['MAPE (%)'].idxmin()]
best_batch = int(best_result['Batch Size'])

print("-" * 60)
print(f"KESIMPULAN AKHIR (METODE TANPA NORMALISASI):")
print(f"Konfigurasi Terbaik Ditemukan:")
print(f"1. Window Size : {FIXED_WINDOW_SIZE}")
print(f"2. Epoch       : {FIXED_EPOCH}")
print(f"3. Batch Size  : {best_batch}")
print(f"MAPE Terendah  : {best_result['MAPE (%)']:.4f}%")
print("="*60)

✓ Data berhasil dimuat.

[Info] Membentuk dataset dengan Window Size TETAP: 90

MULAI UJI COBA SKENARIO C (Mencari Batch Size Terbaik)
Window: 90 | Epoch: 50
KONDISI: TANPA NORMALISASI

[Proses] Training dengan Batch Size: 16 ...
6/6 [==============================] - 5s 152ms/step
   -> Selesai. Hasil MAPE: 57.8064%

[Proses] Training dengan Batch Size: 30 ...
6/6 [==============================] - 9s 324ms/step
   -> Selesai. Hasil MAPE: 84.8998%

[Proses] Training dengan Batch Size: 64 ...
6/6 [==============================] - 2s 65ms/step
   -> Selesai. Hasil MAPE: 95.4120%

   Batch Size   MAPE (%)
0          16  57.806426
1          30  84.899789
2          64  95.412010
------------------------------------------------------------
KESIMPULAN AKHIR (METODE TANPA NORMALISASI):
Konfigurasi Terbaik Ditemukan:
1. Window Size : 90
2. Epoch       : 50
3. Batch Size  : 16
MAPE Terendah  : 57.8064%


UJI COBA 3 LAPISAN MODEL LSTM
DENGAN learning_rate=0.00001

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
import os 
from sklearn.metrics import mean_absolute_percentage_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping

# 1. Load Data
path_file_input = r"E:\Skripsi\Prediksi-bawang-merah\python\data\Data_Clean_Surabaya.xlsx"

if os.path.exists(path_file_input):
    df = pd.read_excel(path_file_input)
    print("✓ Data berhasil dimuat.")
else:
    print("X FILE TIDAK DITEMUKAN!")

raw_data = df['Harga (Rp)'].values.reshape(-1, 1)

print(f"Data siap. Jumlah baris: {len(raw_data)}")
print("Melanjutkan ke Uji Coba Skenario A (3 Layer LSTM)...")

# DEFINISI FUNGSI & ARSITEKTUR

def create_sliding_window(dataset, window_size):
    X, Y = [], []
    for i in range(len(dataset) - window_size):
        X.append(dataset[i:(i + window_size), 0])
        Y.append(dataset[i + window_size, 0])
    return np.array(X), np.array(Y)

def build_model_lstm(input_shape):
    model = Sequential()
    
    # --- MODIFIKASI: MENJADI 3 LAYER LSTM ---
    
    # Layer 1 (Wajib return_sequences=True)
    model.add(LSTM(100, return_sequences=True, input_shape=input_shape))
    model.add(Dropout(0.02))
    
    # Layer 2 (Tengah - return_sequences=True)
    model.add(LSTM(100, return_sequences=True))
    model.add(Dropout(0.02))
    
    # Layer 3 (Terakhir - return_sequences=False)
    model.add(LSTM(100, return_sequences=False))
    model.add(Dropout(0.02))
    
    # Layer Output
    model.add(Dense(25, activation='relu'))
    model.add(Dense(1)) 
    
    # Tetap pakai Konfigurasi "Jurus Terakhir" (LR Kecil & MAE)
    # Karena kita masih pakai data Tanpa Normalisasi
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
                  loss='mean_squared_error')
    return model

# EKSEKUSI UJI COBA SKENARIO A
scenario_windows = [30, 60, 90]

FIXED_EPOCH = 100      
FIXED_BATCH_SIZE = 30 

# Early Stopping
early_stop = EarlyStopping(monitor='loss', patience=10, restore_best_weights=True)

results = []

print("\n" + "="*50)
print("MULAI UJI COBA SKENARIO A (3 LAYER LSTM)")
print("="*50)

for win_size in scenario_windows:
    print(f"\n[Proses] Training Window Size: {win_size}...")
    
    # A. Transformasi Data
    X, Y = create_sliding_window(raw_data, win_size)
    X = np.reshape(X, (X.shape[0], X.shape[1], 1))
    
    # B. Split Data
    train_size = int(len(X) * 0.8)
    X_train, X_test = X[:train_size], X[train_size:]
    y_train, y_test = Y[:train_size], Y[train_size:]
    
    # C. Pelatihan Model
    model = build_model_lstm((X_train.shape[1], 1))
    
    # Callbacks
    model.fit(X_train, y_train, 
              epochs=FIXED_EPOCH, 
              batch_size=FIXED_BATCH_SIZE, 
              callbacks=[early_stop],
              verbose=0)
    
    # D. Evaluasi
    predictions = model.predict(X_test)
    mape = mean_absolute_percentage_error(y_test, predictions) * 100
    
    print(f"   -> Selesai. Hasil MAPE: {mape:.4f}%")
    results.append({'Window Size': win_size, 'MAPE (%)': mape})

# HASIL AKHIR
print("\n" + "="*50)
print("TABEL HASIL PERBANDINGAN (3 LAYER)")
print("="*50)
results_df = pd.DataFrame(results)
print(results_df)

best_result = results_df.loc[results_df['MAPE (%)'].idxmin()]
best_window = int(best_result['Window Size'])

print("-" * 50)
print(f"KESIMPULAN TAHAP 1:")
print(f"Window Size Terbaik: {best_window}")
print(f"MAPE Terendah: {best_result['MAPE (%)']:.4f}%")
print("="*50)

✓ Data berhasil dimuat.
Data siap. Jumlah baris: 1043
Melanjutkan ke Uji Coba Skenario A (3 Layer LSTM)...

MULAI UJI COBA SKENARIO A (3 LAYER LSTM)

[Proses] Training Window Size: 30...
7/7 [==============================] - 13s 125ms/step
   -> Selesai. Hasil MAPE: 99.9451%

[Proses] Training Window Size: 60...
7/7 [==============================] - 22s 172ms/step
   -> Selesai. Hasil MAPE: 99.9511%

[Proses] Training Window Size: 90...
